# DengAI: Predicting Disease Spread

Description: Using environmental data collected by various U.S. Federal Government agencies—from the Centers for Disease Control and Prevention to the National Oceanic and Atmospheric Administration in the U.S. Department of Commerce—can you predict the number of dengue fever cases reported each week in San Juan, Puerto Rico and Iquitos, Peru?

First, we'll take a look at some of the features of the training data

In [49]:
import pandas as pd # Data handling 
import numpy as np

import tensorflow as tf # Neural networks  
from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objects as go # Visualization
import plotly.express as px

import sklearn as sk # Stats / ML
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, TimeSeriesSplit
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import impute

from xgboost import XGBRegressor # Efficient Gradient Boosting

In [54]:
df = pd.read_csv("Data/dengue_features_train.csv", index_col=[0,1,2])
df_labels = pd.read_csv("Data/dengue_labels_train.csv", index_col=[0,1,2])
df_features_test = pd.read_csv("Data/dengue_features_test.csv", index_col=[0,1,2])
submission_format = pd.read_csv("Data/submission_format.csv")

print(df.shape)
print(df_features_test.shape)
display(df_labels.head(2))
display(df.head(2))
display(df_features_test.head(2))


(1456, 21)
(416, 21)


total_cases
city year weekofyear             
sj   1990 18                    4
          19                    5

week_start_date  ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
city year weekofyear                                                          
sj   1990 18              1990-04-30   0.1226  0.103725  0.198483  0.177617   
          19              1990-05-07   0.1699  0.142175  0.162357  0.155486   

                      precipitation_amt_mm  reanalysis_air_temp_k  \
city year weekofyear                                                
sj   1990 18                         12.42             297.572857   
          19                         22.82             298.211429   

                      reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
city year weekofyear                                                       
sj   1990 18                     297.742857                   292.414286   
          19                     298.442857                   293.951429   

                      reanalysis_max_air_temp_k  ...  \
city year weekofyear                             ...   
sj   1990 18                              299.8  ...   
          19                              300.9  ...   

                      reanalysis_precip_amt_kg_per_m2  \
city year weekofyear                                    
sj   1990 18                                    32.00   
          19                                    17.94   

                      reanalysis_relative_humidity_percent  \
city year weekofyear                                         
sj   1990 18                                     73.365714   
          19                                     77.368571   

                      reanalysis_sat_precip_amt_mm  \
city year weekofyear                                 
sj   1990 18                                 12.42   
          19                                 22.82   

                      reanalysis_specific_humidity_g_per_kg  \
city year weekofyear                                          
sj   1990 18                                      14.012857   
          19                                      15.372857   

                      reanalysis_tdtr_k  station_avg_temp_c  \
city year weekofyear                                          
sj   1990 18                   2.628571           25.442857   
          19                   2.371429           26.714286   

                      station_diur_temp_rng_c  station_max_temp_c  \
city year weekofyear                                                
sj   1990 18                         6.900000                29.4   
          19                         6.371429                31.7   

                      station_min_temp_c  station_precip_mm  
city year weekofyear                                         
sj   1990 18                        20.0               16.0  
          19                        22.2                8.6  

[2 rows x 21 columns]

week_start_date  ndvi_ne  ndvi_nw   ndvi_se   ndvi_sw  \
city year weekofyear                                                         
sj   2008 18              2008-04-29  -0.0189  -0.0189  0.102729  0.091200   
          19              2008-05-06  -0.0180  -0.0124  0.082043  0.072314   

                      precipitation_amt_mm  reanalysis_air_temp_k  \
city year weekofyear                                                
sj   2008 18                         78.60             298.492857   
          19                         12.56             298.475714   

                      reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
city year weekofyear                                                       
sj   2008 18                     298.550000                   294.527143   
          19                     298.557143                   294.395714   

                      reanalysis_max_air_temp_k  ...  \
city year weekofyear                             ...   
sj   2008 18                              301.1  ...   
          19                              300.8  ...   

                      reanalysis_precip_amt_kg_per_m2  \
city year weekofyear                                    
sj   2008 18                                    25.37   
          19                                    21.83   

                      reanalysis_relative_humidity_percent  \
city year weekofyear                                         
sj   2008 18                                     78.781429   
          19                                     78.230000   

                      reanalysis_sat_precip_amt_mm  \
city year weekofyear                                 
sj   2008 18                                 78.60   
          19                                 12.56   

                      reanalysis_specific_humidity_g_per_kg  \
city year weekofyear                                          
sj   2008 18                                      15.918571   
          19                                      15.791429   

                      reanalysis_tdtr_k  station_avg_temp_c  \
city year weekofyear                                          
sj   2008 18                   3.128571           26.528571   
          19                   2.571429           26.071429   

                      station_diur_temp_rng_c  station_max_temp_c  \
city year weekofyear                                                
sj   2008 18                         7.057143                33.3   
          19                         5.557143                30.0   

                      station_min_temp_c  station_precip_mm  
city year weekofyear                                         
sj   2008 18                        21.7               75.2  
          19                        22.2               34.3  

[2 rows x 21 columns]

Now, let's take a look at some visualizations of the data. The correlation matrix uses the Pearson correlation coefficient. More info on the documentation of .corr() can be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)

In [55]:
df = df.join(df_labels) #append labels to dataframe
df.reset_index(inplace=True)
df_features_test.reset_index(inplace=True)

print(df.shape)
print(df_features_test.shape)

# Seperate data for San Juan
sj_train_features = df.loc[df['city'] == 'sj', :]
sj_train_labels = sj_train_features['total_cases'].to_frame()

# Separate data for Iquitos
iq_train_features = df.loc[df['city'] == 'iq', :]
iq_train_labels = iq_train_features['total_cases'].to_frame()

sj_train_features.drop('city', axis=1, inplace=True)
iq_train_features.drop('city', axis=1, inplace=True) 

print(df.isnull().sum())

(1456, 25)
(416, 24)
city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                

In [41]:
tc_fig = px.bar(df_labels, x='total_cases', title='Total Cases Distribution')
tc_fig.show()

time_fig = go.Figure(data=go.Scatter(
    x=sj_train_features['week_start_date'],
    y=sj_train_features['total_cases'],
    mode='lines', name='Total Cases for SJ'), 
    layout=go.Layout(xaxis_title='Date', yaxis_title='Number of Cases', title={
        'text':'Cases over Time for sj/iq'
    }),
)

time_fig.add_trace(go.Scatter(
    x=iq_train_features['week_start_date'],
    y=iq_train_features['total_cases'],
    mode='lines', name='Total Cases for IQ'))

time_fig.show()

fig_corr = go.Figure(data=go.Heatmap(
    z=df.corr(),
    x=df.corr().columns,
    y=df.corr().columns,
    hoverongaps=False,
), layout=go.Layout(
    height=900,
    width=900,
    title = {
        'text':'Correlation Matrix',
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    }

))

fig_corr.show()

charts = {'reanalysis_sat_precip_amt_mm':'Precipitation Amount (mm)', 'reanalysis_avg_temp_k':'Average Air Temperature (k)', 
        'reanalysis_relative_humidity_percent':'Mean Relative Humidity (%)',
        'reanalysis_dew_point_temp_k':'Average Dew Point Temperature (k)'
         }

for key in charts:
    data = go.Scatter(x=sj_train_features[key], y=sj_train_features['total_cases'], mode='markers', marker_color=sj_train_features['total_cases'])
    layout = go.Layout(
        title = {
            'text': charts[key] + ' vs Number of Cases',
            'y':0.9,
            'x':0.5,
            'xanchor':'center',
            'yanchor':'top'
        },
        xaxis_title = "Number of Cases",
        yaxis_title = charts[key]
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

charts['total_cases'] = 'Total Cases'
fig = px.scatter_matrix(df, dimensions=list(charts))
fig.show()

From what we can see, the data is fairly Gaussian. The DewPoint data is slightly skewed, but we'll ignore that for now. Next, lets explore how clean the data is.

In [42]:
num_nans = sum([True for idx, row in df.iterrows() if any(row.isnull())]) #count number of rows with ANY NaN value
ratio_nan = num_nans / len(df)

print(str(ratio_nan * 100) + "% of rows have some NaN value\n\n")

d = go.Bar(x=df.columns, y=(df.isnull().sum() / df.shape[0]) * 100)
l = go.Layout(
    title = {
        'text': 'Number of NaN values (%) in columns',
        'y':0.9,
        'x':0.5,
        'xanchor':'center',
        'yanchor':'top'
    },
    xaxis_title = 'Column name',
    yaxis_title = 'Number of NaN values (%)'
)

fig = go.Figure(data=d, layout=l)
fig.show()

17.6510989010989% of rows have some NaN value




As we can see, the percentage of rows with some NaN value is pretty high, so it wouldn't be a great idea to just remove them entirely. Instead, lets try to fill those missing values with some meaningful data. 

I initially tested filling with the mean value, but that increased bias significantly. Let's try using scikit-learn's KNNImputer. The documentation and explanation can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)

In [56]:
print(df.columns)

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'total_cases'],
      dtype='object')


In [57]:
df = pd.get_dummies(df, columns=['city'], drop_first=True)
df_features_test = pd.get_dummies(df_features_test, columns=['city'], drop_first=True)

df.rename(columns={'city_sj':'city'}, inplace=True)
df_features_test.rename(columns={'city_sj':'city'}, inplace=True)

In [58]:
display(df.head(1))

,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,city
0,1990,18,1990-04-30,0.1226,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,12.42,14.012857,2.628571,25.442857,6.9,29.4,20.0,16.0,4,1


In [59]:
df.set_index(keys=['year', 'weekofyear', 'city', 'week_start_date', 'total_cases'], inplace=True)
df_features_test.set_index(keys=['year', 'weekofyear', 'city', 'week_start_date'], inplace=True)

In [60]:
imp = impute.KNNImputer() 
t = pd.DataFrame(imp.fit_transform(df), columns=df.columns, index=df.index)
df = t.copy()

w = pd.DataFrame(imp.fit_transform(df_features_test), columns=df_features_test.columns, index=df_features_test.index)
df_features_test = w.copy()

df.reset_index(inplace=True)
df_features_test.reset_index(inplace=True)
print(df.shape)
print(df_features_test.shape)

# w = pd.DataFrame(imp.fit_transform(df_features_test.iloc[:, 3:]), columns=df_features_test.columns[3:])
# df_features_test = pd.merge(df_features_test, w, on=[i for i in w.columns], how='right')

(1456, 25)
(416, 24)


Now, lets split the dataset by city and drop some highly correlated variables (by the Pearson correlation coefficient)

In [61]:
#Remove some highly correlated columns
# Seperate data for San Juan
sj_train_features = df.loc[df['city'] == 1]
sj_train_labels = sj_train_features['total_cases'].to_frame()

# Separate data for Iquitos
iq_train_features = df.loc[df['city'] == 0]
iq_train_labels = iq_train_features['total_cases'].to_frame()

sj_train_features.drop('city', axis=1, inplace=True)
iq_train_features.drop('city', axis=1, inplace=True) 

df.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)
sj_train_features.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)
iq_train_features.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)
df_features_test.drop(['precipitation_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k'], axis=1, inplace=True)

print(df.isnull().sum()) #should be zero for all columns
print(df_features_test.isnull().sum())
print(sj_train_features.isnull().sum())
print(iq_train_features.isnull().sum())

year                                    0
weekofyear                              0
city                                    0
week_start_date                         0
total_cases                             0
ndvi_ne                                 0
ndvi_nw                                 0
ndvi_se                                 0
ndvi_sw                                 0
reanalysis_air_temp_k                   0
reanalysis_avg_temp_k                   0
reanalysis_dew_point_temp_k             0
reanalysis_max_air_temp_k               0
reanalysis_precip_amt_kg_per_m2         0
reanalysis_relative_humidity_percent    0
reanalysis_sat_precip_amt_mm            0
reanalysis_tdtr_k                       0
station_avg_temp_c                      0
station_diur_temp_rng_c                 0
station_max_temp_c                      0
station_min_temp_c                      0
station_precip_mm                       0
dtype: int64
year                                    0
weekofyear           

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Great. Now, lets try a constructing a baseline model using Linear Regression. Note that we do not have to normalize the data when using ordinary least squares. An explanation can be found [here](https://stats.stackexchange.com/questions/29781/when-conducting-multiple-regression-when-should-you-center-your-predictor-varia).

In [62]:
y = df_labels[['total_cases']]

df_no_y = df.copy()
df_no_y.drop(['total_cases', 'week_start_date'], inplace=True, axis=1)

kf = sk.model_selection.TimeSeriesSplit(n_splits=5) #Use sklearn TimeSeries CV to prevent fitting on "future" data
reg = linear_model.LinearRegression()

train_errors = []
test_errors = []
diff_errors = []

for train_index, test_index in kf.split(df_no_y):
    d_train = df_no_y.iloc[train_index]
    t_train = y.iloc[train_index]
    
    d_test = df_no_y.iloc[test_index]
    t_test = y.iloc[test_index]
    
    reg.fit(d_train, t_train)
    pred_test = reg.predict(d_test)
    pred_train = reg.predict(d_train)
    train_errors.append(sk.metrics.mean_absolute_error(pred_train, t_train))
    test_errors.append(sk.metrics.mean_absolute_error(pred_test, t_test))

for tr, te in zip(train_errors, test_errors):
    diff_errors.append(tr - te)

So, we trained a linear regression model using CV with n=5 in order to reduce variance. Now, let's plot the train and test errors to get a rough estimate about our model's variance (overfitting).

In [63]:
fig = go.Figure()
train_trace = go.Scatter(y=train_errors, mode='lines', name='Train MAE')
test_trace = go.Scatter(y=test_errors, mode='lines', name='Test MAE')
diff_trace = go.Scatter(y=diff_errors, mode='lines', name='Train MAE - Test MAE')
fig.add_trace(train_trace)
fig.add_trace(test_trace)
fig.add_trace(diff_trace)
fig.update_layout(xaxis_title='i-th Fold', yaxis_title='MAE', title={
    'text':'Mean Absolute Errors',
    'y':0.9,
    'x':0.45,
    'xanchor':'center',
    'yanchor':'top'

})

fig.show()
y_pred = reg.predict(df_no_y)

def chart_actual_and_pred(df_t, pred, df_labels, mode, city_name):
    trace_actual = go.Scatter(
        x=df_t['week_start_date'],
        y=df_labels['total_cases'],
        name='Actual Number of Cases',
        mode=mode
    )

    trace_pred = go.Scatter(
        x=df_t['week_start_date'],
        y=pred.ravel(),
        name='Predicted Number of Cases',
        mode=mode
    )

    pred_fig = go.Figure()
    pred_fig.add_trace(trace_actual)
    pred_fig.add_trace(trace_pred)
    pred_fig.update_layout(
        title='Predicted vs Actual Values for ' + city_name,
        xaxis_title='Date',
        yaxis_title='Total Cases',
    )

    pred_fig.show()

chart_actual_and_pred(df, y_pred, df, 'lines', 'Both Cities')

In [64]:
print(df.shape)
print(df_features_test.shape)

df.to_csv("/Users/jmlehrer/DSS/Dengue-Fever-Notebooks/Data/train_imputed.csv", index=False)
df_features_test.to_csv("/Users/jmlehrer/DSS/Dengue-Fever-Notebooks/Data/test_imputed.csv", index=False)

(1456, 22)
(416, 21)


The model did not perform very well with Linear Regression, with an average MAE on the test set of about 23. However, note that we did not train each city seperately for this model. Let's treat this as our baseline.

The current leader on this competition has an MAE of about 10 on the test set, so we have a ways to go.

## Gradient Boosting

Now, let's construct a gradient boosted tree model using XGBoost. As far as hyperparameter optimization, let's start with a GridSearchCV

In [11]:
xgb = XGBRegressor(objective='reg:squarederror') # Try with smooth loss first

sj_features_fit = sj_train_features.copy()
iq_features_fit = iq_train_features.copy()

sj_features_fit.drop(['week_start_date', 'total_cases'], axis=1, inplace=True)
iq_features_fit.drop(['week_start_date', 'total_cases'], axis=1, inplace=True)

In [12]:
param = {
    'max_depth':[i for i in range(3,7)], # depth of tree
    'eta':[0.001, .01], # Step size shrinkage used in update to prevents overfitting
    'gamma':[0, 1], # Minimum loss reduction required to make a further partition on a leaf node of the tree (idk bro)
    'n_estimators':[15, 20], # Number of trees 
    'lambda':[.001, .01], # l2 regularization coefficient
    'alpha':[.001, .01] # l1 regularization coefficient
}

grid_search = GridSearchCV(
    estimator = xgb,
    param_grid = param,
    scoring = 'neg_mean_absolute_error',
    verbose = 1,
    cv = 8,
    n_jobs=-1
)

Now, let's fit the model on this grid search. This might take a while since GridSearchCV is what is called an "**exhaustive search**". This means it tests every *permutation* of the hyperparameters. It's a good thing XGBoost is incredibly efficient, else this process could take a long time.

In [13]:
df_c = df.copy()
df_c.drop(['week_start_date', 'total_cases'], axis=1, inplace=True) #drop string representation of time for training

grid_search = grid_search.fit(df_c, df_labels['total_cases'].values)
grid_pred = grid_search.predict(df_c)

chart_actual_and_pred(df, grid_pred, df_labels, 'lines', 'Total')
print(grid_search.best_params_)
print(abs(grid_search.best_score_))

Fitting 8 folds for each of 128 candidates, totalling 1024 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 340 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 840 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 1017 out of 1024 | elapsed:   36.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 1024 out of 1024 | elapsed:   36.8s finished


{'alpha': 0.001, 'eta': 0.001, 'gamma': 0, 'lambda': 0.001, 'max_depth': 5, 'n_estimators': 15}
17.667207036699565


### Feature Importance

Now that we've tuned our gradient boosted model, let's plot which features it is weighting most heavily. Since initially the values are unscaled, we'll divide each importance by the max(importance) for better plotting.

In [14]:
importances = grid_search.best_estimator_.get_booster().get_score(importance_type='total_gain')

importances = {k: v for k, v in sorted(importances.items(), key=lambda item: item[1])}

largest = max(importances.values())
values = [round((val/largest*100)) for val in importances.values()]
keys = list(importances.keys())

importances_fig = go.Figure()
importances_fig.add_trace(
    go.Bar(x=keys, y=values)
)
importances_fig.update_xaxes(
    autorange="reversed"
)
importances_fig.show()

For this first test, we trained the model on the entirety of the dataset. Now, let's try fitting it to each city specifically and see if it results in a lower bias. 

In [15]:
grid_search_sj = grid_search.fit(sj_features_fit, sj_train_labels['total_cases'].values)
grid_search_iq = grid_search.fit(iq_features_fit, iq_train_labels['total_cases'].values)

Fitting 8 folds for each of 128 candidates, totalling 1024 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 1024 out of 1024 | elapsed:   19.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 8 folds for each of 128 candidates, totalling 1024 fits


[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 728 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 1024 out of 1024 | elapsed:   11.1s finished


In [16]:
sj_pred = grid_search_sj.best_estimator_.predict(sj_features_fit)
iq_pred = grid_search_iq.best_estimator_.predict(iq_features_fit)

display(sj_features_fit.head(5))
display(iq_features_fit.head(5))

chart_actual_and_pred(sj_train_features, sj_pred, sj_train_labels, 'lines', 'SJ (MAE {})'.format(grid_search_sj.best_score_))
chart_actual_and_pred(iq_train_features, iq_pred, iq_train_labels, 'lines', 'IQ (MAE {})'.format(grid_search_iq.best_score_))

print(sk.metrics.mean_absolute_error(sj_pred, sj_train_labels['total_cases'].values))

# #NO WAY THIS IS CORRECT.....
# # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# # # Why are these scores the same? Something is not correct
print(grid_search_sj.best_score_)
# print(grid_search_iq.best_score_)
# # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,1990.0,18.0,0.122600,0.103725,0.198483,0.177617,297.572857,297.742857,292.414286,299.8,32.00,73.365714,12.42,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,1990.0,19.0,0.169900,0.142175,0.162357,0.155486,298.211429,298.442857,293.951429,300.9,17.94,77.368571,22.82,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,1990.0,20.0,0.032250,0.172967,0.157200,0.170843,298.781429,298.878571,295.434286,300.5,26.10,82.052857,34.54,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,1990.0,21.0,0.128633,0.245067,0.227557,0.235886,298.987143,299.228571,295.310000,301.4,13.90,80.337143,15.36,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,1990.0,22.0,0.196200,0.262200,0.251200,0.247340,299.518571,299.664286,295.821429,301.9,12.20,80.460000,7.52,3.014286,28.942857,9.371429,35.0,23.9,5.8


,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
936,2000.0,26.0,0.192886,0.132257,0.340886,0.247200,296.740000,298.450000,295.184286,307.3,43.19,92.418571,25.41,8.928571,26.400000,10.775000,32.5,20.7,3.0
937,2000.0,27.0,0.216833,0.276100,0.289457,0.241657,296.634286,298.428571,295.358571,306.6,46.00,93.581429,60.61,10.314286,26.900000,11.566667,34.0,20.8,55.6
938,2000.0,28.0,0.176757,0.173129,0.204114,0.128014,296.415714,297.392857,295.622857,304.5,64.77,95.848571,55.52,7.385714,26.800000,11.466667,33.0,20.7,38.1
939,2000.0,29.0,0.227729,0.145429,0.254200,0.200314,295.357143,296.228571,292.797143,303.6,23.96,87.234286,5.60,9.114286,25.766667,10.533333,31.5,14.7,30.0
940,2000.0,30.0,0.328643,0.322129,0.254371,0.361043,296.432857,297.635714,293.957143,307.0,31.80,88.161429,62.76,9.500000,26.600000,11.480000,33.3,19.1,4.0


31.258468286890505
-7.157734907991611


Let's test some model stacking 

In [17]:
df_test_c = df_features_test.copy()
df_test_c.drop('week_start_date', axis=1, inplace=True)

display(df_test_c.head(1))
# print(df_c.columns)

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,city_sj
0,2008.0,18.0,-0.0189,-0.0189,0.102729,0.0912,298.492857,298.55,294.527143,301.1,25.37,78.781429,78.6,3.128571,26.528571,7.057143,33.3,21.7,75.2,1.0


## Model Stacking

This section is very preliminary, and should not be taken seriously yet. However, the concept is clear. We will train several models, tuning each using GridSearchCV, then predict the test data and use those predictions as the features in the final XGBoost model.

In [18]:
grid_search_cv_models = []

linear_model_list = {
    'Ridge' : linear_model.Ridge(),
    'Lasso' : linear_model.Lasso(),
    'LARS' : linear_model.LassoLars()
}

svm_model_list = {
    'SVR': sk.svm.SVR(),
}

knn_model_list = {
    'KNN' : sk.neighbors.KNeighborsRegressor(),
}

other_model_list = {
    'Bayes_Ridge' : linear_model.BayesianRidge(),
}

linear_params_grid = {
    'alpha':[.1,.5,1,5,10,100],
}

params_svr = {
    'C':[.5,1,5,10,100],
    'degree':[1,3,5],
    'epsilon':[.01, .1, .5]
}

params_knn = {
    'n_neighbors':[1,2,3]
}

learned_features = pd.DataFrame()
learned_features_test = pd.DataFrame()

for model in linear_model_list:
    g_s = GridSearchCV(
        estimator=linear_model_list[model],
        param_grid=linear_params_grid,
        scoring='neg_mean_absolute_error',
        cv=10,
        verbose=1
    )
    g_s.fit(df_c, df_labels['total_cases'].values)
    learned_features['{}_pred'.format(model)] = g_s.predict(df_c)
    learned_features_test['{}_pred'.format(model)] = g_s.predict(df_test_c)
    grid_search_cv_models.append(g_s.best_estimator_)

g_s = GridSearchCV(
    estimator=svm_model_list['SVR'],
    param_grid=params_svr,
    scoring='neg_mean_absolute_error',
    cv=10,
    verbose=1
)

g_s.fit(df_c, df_labels['total_cases'].values)
learned_features['{}_pred'.format('SVR')] = g_s.predict(df_c)
learned_features_test['{}_pred'.format('SVR')] = g_s.predict(df_test_c)
grid_search_cv_models.append(g_s.best_estimator_)

g_s.param_grid = params_knn
g_s.estimator = knn_model_list['KNN']
g_s.fit(df_c, df_labels['total_cases'].values)
learned_features['{}_pred'.format('KNN')] = g_s.predict(df_c)
learned_features_test['{}_pred'.format('KNN')] = g_s.predict(df_test_c)
grid_search_cv_models.append(g_s.best_estimator_)

m = other_model_list['Bayes_Ridge']
m.fit(df_c, df_labels['total_cases'].values)
learned_features['{}_pred'.format('Bayes_Ridge')] = m.predict(df_c)
learned_features_test['{}_pred'.format('Bayes_Ridge')] = m.predict(df_test_c)

grid_search_cv_models.append(m)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:   43.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished


In [19]:
t_m = XGBRegressor(
    objective='reg:squarederror'
)

ts = TimeSeriesSplit()
f_y = df_labels['total_cases']
cv_test_err = []

for train_index, test_index in ts.split(learned_features):
    d_train = learned_features.iloc[train_index]
    t_train = f_y.iloc[train_index]
    
    d_test = learned_features.iloc[test_index]
    t_test = f_y.iloc[test_index]
    
    t_m.fit(d_train, t_train)
    pred_test = t_m.predict(d_test)
    cv_test_err.append(sk.metrics.mean_absolute_error(pred_test, t_test))


print(np.mean(cv_test_err))

# t_gs = t_gs.fit(learned_features, df_labels['total_cases'].values)
# stack_pred = t_gs.predict(learned_features)
# print(t_gs.best_params_)

# data = [
#     go.Scatter(x=df['week_start_date'], y=df['total_cases']),
#     go.Scatter(x=df['week_start_date'], y=stack_pred),
# ]

# p = go.Figure(data=data)
# p.show()

# print(sk.metrics.mean_absolute_error(stack_pred, df['total_cases'].values))

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



13.941878406050776


In [20]:
sj_test = df_test_c[df_test_c['city_sj'] == 1]
iq_test = df_test_c[df_test_c['city_sj'] == 0]

sj_test.drop('city_sj', axis=1, inplace=True)
iq_test.drop('city_sj', axis=1, inplace=True)

print(sj_test.columns == sj_features_fit.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
pred_sj = grid_search_sj.predict(sj_test)
pred_iq = grid_search_iq.predict(iq_test)

In [22]:
sj_test['total_cases'] = pred_sj.ravel()
iq_test['total_cases'] = pred_iq.ravel()

sj_test = sj_test.append(iq_test)
print(sj_test.shape[0] == submission_format.shape[0])

True


/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jmlehrer/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
subm = pd.read_csv("Data/submission_format.csv")
subm.drop('total_cases', axis=1, inplace=True)

In [24]:
merged = pd.merge(subm, sj_test, on=['year', 'weekofyear'])

In [25]:
merged['total_cases'] = merged['total_cases'].apply(lambda x: int(round(x)))

In [26]:
merged = merged[['city','year', 'weekofyear', 'total_cases']]

display(merged.head(500))
subm2 = pd.read_csv("Data/submission_format.csv")
display(subm2.head(500))
merged.to_csv('test.csv', index=False)

,city,year,weekofyear,total_cases
0,sj,2008,18,4
1,sj,2008,19,9
2,sj,2008,20,4
3,sj,2008,21,9
4,sj,2008,22,4
...,...,...,...,...
495,iq,2012,17,9
496,iq,2012,17,8
497,sj,2012,18,9
498,sj,2012,18,7


,city,year,weekofyear,total_cases
0,sj,2008,18,0
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,0
4,sj,2008,22,0
...,...,...,...,...
411,iq,2013,22,0
412,iq,2013,23,0
413,iq,2013,24,0
414,iq,2013,25,0
